## Ping Pong Agents

This is a simple test to demonstrate 2 agents communicating with each other using a session manager.

### Step 1: Register new message body

In this test, we will be using a custom message body type `system.ping` and `system.pong`.

To do so, we need to register the body types and overwrite the default `MessagePydantic` type.



In [1]:
from typing import Literal
from pydantic import BaseModel
from gai.messages import register_body, get_message_cls


@register_body
class PingBodyPydantic(BaseModel):
    type: Literal["system.ping"] = "system.ping"
    content: str


@register_body
class PongBodyPydantic(BaseModel):
    type: Literal["system.pong"] = "system.pong"
    content: str


MessagePydantic = get_message_cls()


### Step 2: Startup Session Manager

We need to start a session manager specific to the new `MessagePydantic` type by calling the factory function `make_session_manager`.

In [2]:
from gai.sessions.session_manager import make_session_manager

SessionManager = make_session_manager(MessagePydantic)

dm = SessionManager(logger_name="pingpong", file_path="dialogue.json")


### Step 3: Create Pinger Agent

Create a `Pinger` agent that will send a `system.ping` message to the `Ponger` agent when the user calls the `ping` method.



In [3]:
import asyncio
from typing import Callable

class Pinger:
    def __init__(
        self,
        node_name: str,
        session_mgr: SessionManager,
        timeout=2,
        pong_cb: Callable = None,
    ):
        self.node_name = node_name
        self.session_mgr = session_mgr
        self.timeout = timeout
        self.pong_cb = pong_cb

    async def subscribe(self):
        # When pinger received a pong message, it will call the pong callback

        await self.session_mgr.subscribe(
            subject="system.pong", callback={"Pinger": self.pong_cb}
        )

    async def ping(self):
        # Send a ping message to the network

        await self.session_mgr.publish(
            type="system.ping", sender="pinger", recipient="ponger", body=self.node_name
        )

        # Wait for a pong response

        await asyncio.sleep(self.timeout)


### Step 4: Create Ponger Agent

Create a `Ponger` agent that will respond with `system.pong` when it receives a `system.ping` message from the `Pinger` agent.




In [4]:
class Ponger:
    def __init__(
        self,
        node_name: str,
        session_mgr: SessionManager,
        timeout=2,
        ping_cb: Callable = None,
    ):
        self.node_name = node_name
        self.session_mgr = session_mgr
        self.timeout = timeout
        self.ping_cb = ping_cb

    async def ping_handler(self, msg: MessagePydantic):
        # When ponger received a ping message, it will send a pong response

        await self.session_mgr.publish(
            type="system.pong",
            sender=self.node_name,
            recipient=msg.header.sender,
            body=self.node_name,
        )

        # Call the ping callback if it is set, eg. to notify the pinger that a ping message was received

        if self.ping_cb:
            await self.ping_cb(msg)

    async def subscribe(self):
        # When ponger received a ping message, it will call the ping handler

        await self.session_mgr.subscribe(
            subject="system.ping", callback={"Ponger": self.ping_handler}
        )


### Step 5: Run Session

The Ponger class is subscribed to the session like this:

```python
        await self.session_mgr.subscribe(
            subject="system.ping", callback={"Ponger": self.ping_handler}
        )
```

That means that when a `system.ping` message is received, the `ping_handler` method will be called and publishes a `system.pong` message.

```python
    async def ping_handler(self, msg: MessagePydantic):
        # When ponger received a ping message, it will send a pong response

        await self.session_mgr.publish(
            type="system.pong",
            sender=self.node_name,
            recipient=msg.header.sender,
            body=self.node_name,
        )
```



In [5]:
# Setup session
session_mgr = SessionManager(
    logger_name="pinger",
    reset=True,
)

await session_mgr.start()

# Register pong
async def ping_handler(msg):
    print(f"Received ping from {msg.header.sender}")


ponger = Ponger(node_name="ponger", session_mgr=session_mgr, ping_cb=ping_handler)
await ponger.subscribe()


# Register ping
async def pong_handler(msg):
    print(f"Received pong from {msg.header.sender}")
pinger = Pinger(node_name="pinger", session_mgr=session_mgr, pong_cb=pong_handler)
await pinger.subscribe()

# Start pinging
await pinger.ping()


Received ping from pinger
Received pong from ponger
